# 기본 정보 입력

In [1]:
# 패키지 불러오기
import openai
import yfinance as yf
import json


# Custom Functions 생성

In [2]:
def get_stock_price(symbol):
    stock = yf.Ticker(symbol)
    price = stock.info['currentPrice']
    return price

In [3]:
def get_latest_company_news(symbol):
    stock = yf.Ticker(symbol)
    news = stock.news
    # 최신 뉴스 3개 리스트에 저장하기
    news_list = []
    num =1
    for item in news[:3]:
        news_list.append(f"{num}: title : "+item['title']+", publisher :"+item['publisher']+", link :"+item['link'])
        num+=1
    return news_list

In [4]:
tools_list = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_price",
            "description": "Retrieve the latest closing price of a stock using its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "The ticker symbol of the stock"
                    }
                },
                "required": ["symbol"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_latest_company_news",
            "description": "Fetches the latest news articles related to a specified company",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {
                        "type": "string",
                        "description": "The ticker symbol of the stock"
                    }
                },
                "required": ["symbol"]
            }
        }
    }
]

# Create Assistant
- instructions: 어시스턴트와 모델이 어떻게 행동하거나 응답해야 하는지 알려줍니다.
- model: 미세 조정된 모델을 포함하여 모든 GPT-3.5 또는 GPT-4 모델을 지정할 수 있습니다. 검색 도구에는 gpt-3.5-turbo-1106 및 gpt-4-1106-preview 모델이 필요합니다.
- tools: API는 OpenAI에서 빌드 및 호스팅하는 코드 인터프리터와 웹 검색을 지원합니다. 또한 함수 호출 기능과 유사한 동작으로 사용자 정의 함수 서명을 정의할 수 있습니다


In [5]:
# API 키 지정하여 클라이언트 선언하기
client = openai.OpenAI(api_key = "발급받은 OpenAI API 키")

In [6]:
instruction = '''
너는 주식 분석 전문가야

규칙:
1. 사용자가 회사 또는 종목에 대해 물어보면, function을 활용해서 회사의 주식 가격과 최신 뉴스를 답해줘.
2. 모든 답변은 한국어로 해줘
'''

# Add the file to the assistant
assistant = client.beta.assistants.create(
  name = "Company Stock Analyst",
  instructions=instruction,
  model="gpt-3.5-turbo",
  tools=tools_list,
)

print(assistant.id)

asst_2kZK6KVTSwKpY8o7KKwAz9ik


In [18]:
print(assistant)

Assistant(id='asst_3EgCcFCcsSYK2twUhDpGm0hH', created_at=1710855525, description=None, file_ids=[], instructions="\nYou are a company stock analyst\n\nRules:\n1. When a user asks about a company, answer them using the company's stock price and recent news.\n2. answer all questions in Korean.\n", metadata={}, model='gpt-3.5-turbo-1106', name='Company Stock Analyst', object='assistant', tools=[ToolFunction(function=FunctionDefinition(name='get_stock_price', description='Retrieve the latest closing price of a stock using its ticker symbol', parameters={'type': 'object', 'properties': {'symbol': {'type': 'string', 'description': 'The ticker symbol of the stock'}}, 'required': ['symbol']}), type='function'), ToolFunction(function=FunctionDefinition(name='get_latest_company_news', description='Fetches the latest news articles related to a specified company', parameters={'type': 'object', 'properties': {'symbol': {'type': 'string', 'description': 'The ticker symbol of the stock'}}, 'required'

# Tread

## Tread 생성하기

In [7]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_PgRjGhEk1PgyYcKCekmThpFh', created_at=1711010060, metadata={}, object='thread')


## Tread 에 메세지 넣기

In [8]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="테슬라의 최근 주가와 최신 뉴스에 대해 알려줘"
)

# Assistant Run 하기

In [9]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id)

print(run.id)

run_efPUuBlv99i6SSpKieoBfMF5


In [10]:
import time

while run.status not in ["completed", "failed", "requires_action"]:
  
  run = client.beta.threads.runs.retrieve(
    thread_id= thread.id,
    run_id= run.id
  )
  print(run.status)
  time.sleep(5)

in_progress
requires_action


In [12]:
tools_to_call

[RequiredActionFunctionToolCall(id='call_L0sJQxuuK6zO8GIOzDtt40qm', function=Function(arguments='{"symbol": "TSLA"}', name='get_stock_price'), type='function'),
 RequiredActionFunctionToolCall(id='call_0658AeGnppCFPLvbzi7b6iMZ', function=Function(arguments='{"symbol": "TSLA"}', name='get_latest_company_news'), type='function')]

In [13]:
tools_to_call = run.required_action.submit_tool_outputs.tool_calls
print("필요 호출 갯수: ", len(tools_to_call))
print(tools_to_call)

TypeError: can only concatenate str (not "int") to str

In [14]:
print(tools_to_call[0].function.name)
print(tools_to_call[0].function.arguments)
print(tools_to_call[1].function.name)
print(tools_to_call[1].function.arguments)

get_stock_price
{"symbol": "TSLA"}
get_latest_company_news
{"symbol": "TSLA"}


In [15]:
tools_output_array = []
for each_tool in tools_to_call:
    tool_call_id = each_tool.id
    function_name = each_tool.function.name
    function_arg = each_tool.function.arguments
    # Json 포맷팅
    function_arg = json.loads(each_tool.function.arguments)
    if (function_name == 'get_stock_price'):
        ## Custom 함수 생성 ##
        output=get_stock_price(function_arg["symbol"])
    if (function_name == 'get_latest_company_news'):
        ## Custom 함수 생성 ##
        output=get_latest_company_news(function_arg["symbol"])
    tools_output_array.append({"tool_call_id": tool_call_id, "output": json.dumps(output)})

print(tools_output_array)

[{'tool_call_id': 'call_L0sJQxuuK6zO8GIOzDtt40qm', 'output': '175.66'}, {'tool_call_id': 'call_0658AeGnppCFPLvbzi7b6iMZ', 'output': '["1: title : China\\u2019s booming EV market puts these 3 companies on the path to growth. Here\\u2019s what sets them apart\\u2014and the challenges they have to face, publisher :Fortune, link :https://finance.yahoo.com/news/china-booming-ev-market-puts-230000468.html", "2: title : Great News for Tesla Stock Investors!, publisher :Motley Fool, link :https://finance.yahoo.com/m/850755a4-c83e-320b-8c81-21332f747336/great-news-for-tesla-stock.html", "3: title : Massive Warning for Rivian, Lucid, and Even Tesla Investors, publisher :Motley Fool, link :https://finance.yahoo.com/m/059639bc-a6eb-3655-bfd2-2a478d6b5342/massive-warning-for-rivian%2C.html"]'}]


In [29]:
tools_output_array

[{'tool_call_id': 'call_wmb1RVogXIYZhOgB1eM0TbEO', 'output': '171.27'},
 {'tool_call_id': 'call_jMzDXblHr8uGMSzjtPZVmQMt',
  'output': '["1: title : Where Will Tesla Stock Be in 3 Years?, publisher :Motley Fool, link :https://finance.yahoo.com/m/8597e275-3ebc-3402-a126-2e3c0eee4dda/where-will-tesla-stock-be-in.html", "2: title : Tesla Stock Is Falling, CEO Elon Musk Gives Revealing Interview. What to Know., publisher :Barrons.com, link :https://finance.yahoo.com/m/28b8d0cd-be77-3922-95fb-0b14878c7a58/tesla-stock-is-falling%2C-ceo.html", "3: title : Dow Jones Futures Fall: Nvidia CEO Speech In Focus; Google, Tesla Surge Ahead Of Fed Meeting, publisher :Investor\'s Business Daily, link :https://finance.yahoo.com/m/6171e83e-7dae-330e-b9bf-caef5b72d2c3/dow-jones-futures-fall%3A.html"]'}]

In [16]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id = thread.id,
    run_id = run.id,
    tool_outputs=tools_output_array
)

In [17]:
while run.status not in ["completed", "failed", "requires_action"]:
  run = client.beta.threads.runs.retrieve(
    thread_id = thread.id,
    run_id = run.id
  )

  print(run.status)
  time.sleep(5)

in_progress
completed


In [18]:
# 메세지 리스트 가져오기
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

#메세지 프린트
for each in messages:
  print(each.role + ": "+each.content[0].text.value)
  print("=========")

assistant: 테슬라의 최근 주가는 175.66달러입니다. 최신 뉴스는 다음과 같습니다:
1. 제목: "China’s booming EV market puts these 3 companies on the path to growth. Here’s what sets them apart­­and the challenges they have to face", 발행사: Fortune, 링크: [기사 링크](https://finance.yahoo.com/news/china-booming-ev-market-puts-230000468.html)
2. 제목: "Great News for Tesla Stock Investors!", 발행사: Motley Fool, 링크: [기사 링크](https://finance.yahoo.com/m/850755a4-c83e-320b-8c81-21332f747336/great-news-for-tesla-stock.html)
3. 제목: "Massive Warning for Rivian, Lucid, and Even Tesla Investors", 발행사: Motley Fool, 링크: [기사 링크](https://finance.yahoo.com/m/059639bc-a6eb-3655-bfd2-2a478d6b5342/massive-warning-for-rivian%2C.html)
user: 테슬라의 최근 주가와 최신 뉴스에 대해 알려줘


In [24]:
messages.data[0].content[0].text.value

'테슬라의 최근 주가는 175.66달러입니다. 최신 뉴스는 다음과 같습니다:\n1. 제목: "China’s booming EV market puts these 3 companies on the path to growth. Here’s what sets them apart\xad\xadand the challenges they have to face", 발행사: Fortune, 링크: [기사 링크](https://finance.yahoo.com/news/china-booming-ev-market-puts-230000468.html)\n2. 제목: "Great News for Tesla Stock Investors!", 발행사: Motley Fool, 링크: [기사 링크](https://finance.yahoo.com/m/850755a4-c83e-320b-8c81-21332f747336/great-news-for-tesla-stock.html)\n3. 제목: "Massive Warning for Rivian, Lucid, and Even Tesla Investors", 발행사: Motley Fool, 링크: [기사 링크](https://finance.yahoo.com/m/059639bc-a6eb-3655-bfd2-2a478d6b5342/massive-warning-for-rivian%2C.html)'

# Assistants 삭제

In [33]:
response = client.beta.assistants.delete(assistant.id)
print(response)

AssistantDeleted(id='asst_3EgCcFCcsSYK2twUhDpGm0hH', deleted=True, object='assistant.deleted')
